In [ ]:
!pip install trimesh pyrender torch torchvision hdbscan Pillow matplotlib numpy-stl opencv-python scikit-learn pyrender


In [ ]:
import os
import shutil
import numpy as np
import hdbscan
import torch
from torch import nn
from torchvision import models, transforms
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from stl import mesh as np_mesh  # Import numpy-stl


In [ ]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"

In [ ]:

# Paths
input_folder = ''  # Path to your STL files
output_folder = ''  # Output folder for clustered files
features_folder = ''  # Folder to save features
os.makedirs(output_folder, exist_ok=True)
os.makedirs(features_folder, exist_ok=True)

# Function to load STL file using numpy-stl and generate projections using Matplotlib
def generate_projections(stl_file):
    try:
        # Load STL file with numpy-stl
        your_mesh = np_mesh.Mesh.from_file(stl_file)
        print(f"Loaded {stl_file}")  # Debug print

        # Extract vertices and faces
        vertices = your_mesh.vectors.reshape(-1, 3)
        faces = np.arange(len(vertices)).reshape(-1, 3)

        # Create a figure for plotting
        fig = plt.figure(figsize=(4, 4))

        # Isometric view
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='gray', edgecolor='none')
        ax.view_init(elev=30, azim=45)
        plt.axis('off')
        fig.tight_layout(pad=0)
        fig.canvas.draw()
        iso_view = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.clf()

        # Orthographic views
        ortho_views = []
        for elev, azim in [(90, 0), (0, 0), (0, 90)]:
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='gray', edgecolor='none')
            ax.view_init(elev=elev, azim=azim)
            plt.axis('off')
            fig.tight_layout(pad=0)
            fig.canvas.draw()
            ortho_view = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (3,))
            ortho_views.append(ortho_view)
            plt.clf()

        plt.close(fig)
        print(f"Generated projections for {stl_file}")  # Debug print
        return iso_view, ortho_views
    except Exception as e:
        print(f"Error processing {stl_file}: {e}")
        return None, None

# Load pre-trained SqueezeNet and modify it
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.squeezenet = models.squeezenet1_1(pretrained=True)
        self.features = nn.Sequential(*list(self.squeezenet.children())[:-1])

    def forward(self, x):
        return self.features(x).view(x.size(0), -1)

# Move the model to CPU
model = FeatureExtractor().eval()

# Transform for input to SqueezeNet
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Extract features
stl_files = [f for f in os.listdir(input_folder) if f.endswith('.stl')]
all_features = []

for i, stl_file in enumerate(stl_files, 1):  # Start enumeration at 1
    file_path = os.path.join(input_folder, stl_file)

    # Generate projections
    iso_view, ortho_views = generate_projections(file_path)
    if iso_view is None or ortho_views is None:
        continue  # Skip if there was an error in generating projections

    # Extract features from projections
    projections = [iso_view] + ortho_views
    features = []
    for proj in projections:
        input_tensor = preprocess(proj).unsqueeze(0)  # Keep on CPU
        with torch.no_grad():
            feature = model(input_tensor).numpy().flatten()  # Keep on CPU
            features.append(feature)

    combined_features = np.concatenate(features)
    all_features.append(combined_features)

    # Save the features with the same name as the STL file
    feature_file_name = os.path.join(features_folder, f"{os.path.splitext(stl_file)[0]}_features.npy")
    np.save(feature_file_name, combined_features)

    # Print progress
    print(f"Processed and saved features for {i}/{len(stl_files)}: {stl_file}")

